In [1]:
from elevenlabs import generate, play, stream
from elevenlabs import set_api_key
from dotenv import load_dotenv
import os
from openai import OpenAI
import base64
import cv2
import time
from IPython.display import display, clear_output
import ipywidgets as widgets
client = OpenAI()
load_dotenv()
set_api_key(os.getenv("eleven_labs_api_key"))

In [2]:
message_history = [{
                  "role": "system",
                  "content": "You are a narrator, you will be given images then you have to narrate it in the tone of David Attenborough. If there has been previous responses then make the narration flow to the next scene. Always just start narrating don't say anything else. The narrations per response should be short like 50 words. Remember to make the narration flow from the prior if there were a prior narration."
                }]
assistant = ""

In [3]:
def analyze_image(image):
  def encode_image(image_path):
    with open(image_path, "rb") as image_file:
      return base64.b64encode(image_file.read()).decode('utf-8')
  message = {
        "role": "user",
        "content": [
          {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encode_image(image)}"
              }
          },
        ],
      }
  message_history.append(message)
  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=message_history,
    max_tokens=100,
    stream=True
  )
  message_history[-1]['content'].pop()
  return response

In [4]:
def text_iterator(response):
  global assistant
  for chunk in response:
      delta = chunk.choices[0].delta.content
      if delta is not None:
          assistant += delta + ""
          print(delta, end="")
          yield delta
      else:
         pass

In [5]:
def narrate():
    global assistant
    cap = cv2.VideoCapture(0)
    time.sleep(1)
    cnt = 0
    try:
        while True:
            ret, frame = cap.read()
            if ret:
                # _, jpeg_frame = cv2.imencode('.jpg', frame)
                # image_widget = widgets.Image(value=jpeg_frame.tobytes(), format='jpg')
                # display(image_widget)
                # clear_output(wait=True)
                
                filename = f"image_{cnt}.png"
                cv2.imwrite(filename, frame)
                print(f"Captured {filename}")
                msg = analyze_image(filename)
                audio_stream = generate(
                  text=text_iterator(response=msg),
                  stream=True,
                  voice="6X5tLNVVSm3Hj6AjopAd"
                )
                stream(audio_stream)
                print()
                message = {
                  "role": "assistant",
                  "content": assistant
                }
                message_history.append(message)
                assistant = ""
                cnt += 1
            else:
                print("Failed to capture")
                break

    except KeyboardInterrupt:
        print("Stopped by user")
    finally:
        cap.release()
        cv2.destroyAllWindows()

narrate()


2023-12-03 19:18:16.620 python[24187:668582] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Captured image_0.png
In the modern jungle of technology and concrete, we find the Homo sapiens in a common habitat, the office. Here, a young specimen engages with her electronic tool, her gaze focused, pondering the virtual landscape before her. It's a still moment of contemplation in the busy ecosystem of work.
Captured image_1.png
Ah, a delightful turn of events. The young Homo sapiens, clearly buoyed by social interaction, displays a gesture of joy - the smile. This expression of happiness is a universal language among this species, signaling camaraderie and a momentary respite from the day's toils.
Captured image_2.png
Here we see the human engaging in a ritual of hydration. The metal container, a modern evolution of the simple cup, is a tool favored by this species for maintaining vital water intake. Her expression, still cheerful, communicates the satisfaction such a simple act can bring amidst the complexities of her environment.
Captured image_3.png
Ah, the human engages in a 